# Harsha Equation — Advanced QuTiP Notebook

This notebook constructs three Harsha-model systems (two-site exciton, minimal radical-pair, and enzyme-tunneling toy), builds finite-time dynamical maps by evolving operator-basis elements under a self-consistent time-local solver, computes Choi eigenvalues (CPTP test), plots von Neumann entropy (proxy for entropy production), computes a coherent-information proxy, and reports simple stability diagnostics and a parameter-fit stub.

Outputs (CSV and PNG) are saved to `./harsha_results/`.


In [1]:
# CELL 1: Self-consistent solver and diagnostics (idempotent)
import os
import numpy as np
import pandas as pd
import qutip as qt
import matplotlib.pyplot as plt
from scipy.linalg import sqrtm
from numpy.linalg import eigvals, norm

# Ensure OUTDIR exists (OUTDIR defined in earlier cell)
if 'OUTDIR' not in globals():
    OUTDIR = "./harsha_results"
os.makedirs(OUTDIR, exist_ok=True)

def vec_qobj(qobj):
    return qt.operator_to_vector(qobj).full().flatten()

def evolve_basis_self_consistent(H0, proj_observable, c_ops_base, tlist, lambda0=0.0, kappa=0.0):
    """
    Build finite-time superoperator Phi by evolving each basis operator
    under time-local mean-field update H(t) = H0 + (lambda0 + kappa * Tr[proj rho]) * proj_observable.

    This version enforces QuTiP `dims` consistency for H0, proj_observable and c_ops.
    It builds the operator basis explicitly with matching dims to avoid the
    incompatible-dimensions errors.
    """
    # ensure inputs are Qobj
    if not isinstance(H0, qt.Qobj):
        H0 = qt.Qobj(H0)
    dims = H0.dims
    d = H0.shape[0]

    # ensure proj_observable is a Qobj with matching dims
    if not isinstance(proj_observable, qt.Qobj):
        proj_observable = qt.Qobj(proj_observable)
    if proj_observable.dims != dims:
        proj_observable = proj_observable.copy()
        proj_observable.dims = dims

    # ensure collapse operators have matching dims
    c_ops = []
    for L in c_ops_base:
        if not isinstance(L, qt.Qobj):
            Lq = qt.Qobj(L)
        else:
            Lq = L.copy()
        if Lq.dims != dims:
            Lq.dims = dims
        c_ops.append(Lq)

    # build basis operators with explicit dims (|i><j| basis)
    basis_ops = []
    for i in range(d):
        for j in range(d):
            M = np.zeros((d, d), dtype=complex)
            M[i, j] = 1.0
            basis_ops.append(qt.Qobj(M, dims=dims))

    evolved = []
    for B in basis_ops:
        rho = B.copy()
        # time-step integration with updated H at each small step
        for idx in range(len(tlist) - 1):
            dt = float(tlist[idx + 1] - tlist[idx])
            # ensure rho is Qobj with right dims (safety)
            if not isinstance(rho, qt.Qobj):
                rho = qt.Qobj(rho, dims=dims)
            if rho.dims != dims:
                rho = rho.copy(); rho.dims = dims

            # compute population / expectation (Tr[rho * proj])
            try:
                pop = (rho * proj_observable).tr()
            except Exception:
                # fallback: try expectation
                pop = qt.expect(proj_observable, rho) if hasattr(qt, "expect") else 0.0

            lam = float(lambda0 + kappa * np.real(pop))
            Ht = H0 + lam * proj_observable

            try:
                res = qt.mesolve(Ht, rho, [0.0, dt], c_ops, [])
                rho = res.states[-1]
            except Exception as e:
                # fallback: keep rho unchanged but warn (don't crash the sweep)
                # You can replace this with logging if you prefer.
                # print("mesolve failed at small step (keeping state):", e)
                rho = rho

        # store full matrix (as numpy array) for later vectorization
        evolved.append(rho.full())

    # assemble Phi as d^2 x d^2 (column-major mapping vec(in)->vec(out))
    Phi = np.zeros((d * d, d * d), dtype=complex)
    for col, mat in enumerate(evolved):
        # create Qobj with dims so operator_to_vector uses correct dims
        out_q = qt.Qobj(mat, dims=dims)
        vec_out = qt.operator_to_vector(out_q).full().flatten()
        Phi[:, col] = vec_out

    return Phi



def superop_to_choi(Phi, d):
    return Phi.reshape((d,d,d,d)).transpose((0,2,1,3)).reshape((d*d,d*d))

def min_choi_eigenvalue(Phi, d):
    J = superop_to_choi(Phi, d)
    eigs = np.linalg.eigvals(J)
    return np.min(np.real(eigs)), eigs

def entropy_trajectory(H0, proj_obs, c_ops, tlist, lambda0, kappa, rho0):
    rho = qt.Qobj(rho0)
    S_list = [qt.entropy_vn(rho)]
    for idx in range(len(tlist)-1):
        dt = tlist[idx+1]-tlist[idx]
        pop = np.real((rho * proj_obs).tr())
        lam = lambda0 + kappa*pop
        Ht = H0 + lam*proj_obs
        res = qt.mesolve(Ht, rho, [0, dt], c_ops, [])
        rho = res.states[-1]
        S_list.append(qt.entropy_vn(rho))
    return np.array(S_list)

def relative_entropy(rho, sigma):
    try:
        return qt.entropy_relative(qt.Qobj(rho), qt.Qobj(sigma))
    except Exception:
        # fallback crude computation (diagonal basis)
        r = np.real(qt.Qobj(rho).full().diagonal())
        s = np.real(qt.Qobj(sigma).full().diagonal())
        mask = r > 1e-12
        eps = 1e-12
        return np.sum(r[mask] * (np.log(r[mask]) - np.log(np.maximum(s[mask], eps))))

def entropy_production_proxy(H0, proj_obs, c_ops, tlist, lambda0, kappa, rho0, rho_inst_fn=None):
    rho = qt.Qobj(rho0)
    rho_list = [rho]
    for idx in range(len(tlist)-1):
        dt = tlist[idx+1]-tlist[idx]
        pop = np.real((rho * proj_obs).tr())
        lam = lambda0 + kappa*pop
        Ht = H0 + lam*proj_obs
        res = qt.mesolve(Ht, rho, [0, dt], c_ops, [])
        rho = res.states[-1]
        rho_list.append(rho)
    # build instantaneous reference states
    if rho_inst_fn is None:
        try:
            ss = qt.steadystate(H0, c_ops)
            rho_inst = [ss]*len(rho_list)
        except Exception:
            rho_inst = [rho_list[0]]*len(rho_list)
    else:
        rho_inst = [rho_inst_fn(t) for t in np.linspace(0,1,len(rho_list))]
    Srel = np.array([relative_entropy(rho_list[i], rho_inst[i]) for i in range(len(rho_list))])
    times = np.linspace(0,1,len(rho_list))
    dS = -np.diff(Srel) / np.diff(times)
    return times[1:], dS

def coherent_information_proxy(Phi, d, samples=100):
    best = 1e9
    for _ in range(samples):
        psi = qt.rand_ket(d)
        rho = psi * psi.dag()

        # vectorize input operator
        rho_vec = qt.operator_to_vector(rho)

        # apply channel superoperator
        out_vec = Phi.dot(rho_vec.full().flatten())

        # reconstruct as operator-ket Qobj with correct dims
        out_vec_qobj = qt.Qobj(out_vec.reshape((d*d, 1)),
                               dims=[[[d],[d]], [1,1]],
                               superrep="super")

        # map back to operator
        out = qt.vector_to_operator(out_vec_qobj)

        # entropy of output
        S_out = qt.entropy_vn(out)
        if S_out < best:
            best = S_out
    return -best





def spectral_gap_of_lindbladian(H, c_ops):
    L = qt.liouvillian(H, c_ops)
    eigs = eigvals(L.full())
    desc = np.sort(np.real(eigs))[::-1]
    if len(desc) > 1:
        gap = desc[0] - desc[1]
    else:
        gap = 0.0
    return gap, eigs

def fit_kappa_from_observables(H0, proj_obs, c_ops_base, tlist, rho0, measured_eta_t, kappa_init=0.1):
    def loss(kappa_arr):
        k = float(kappa_arr[0])
        rho = qt.Qobj(rho0)
        eta_model = []
        for idx in range(len(tlist)-1):
            dt = tlist[idx+1] - tlist[idx]
            pop = np.real((rho * proj_obs).tr())
            lam = 0.0 + k * pop
            Ht = H0 + lam * proj_obs
            res = qt.mesolve(Ht, rho, [0, dt], c_ops_base, [])
            rho = res.states[-1]
            eta_model.append(np.real((rho * proj_obs).tr()))
        eta_model = np.array(eta_model)
        measured = np.array(measured_eta_t)[:len(eta_model)]
        return np.sum((eta_model - measured)**2)
    res = minimize(lambda x: loss(x), x0=[kappa_init], bounds=[(0,5)])
    return res


In [2]:
# CELL 1b: Model builders (needed by later cells)

def build_two_site(eps1=0.0, eps2=0.1, J=0.05):
    d = 2
    proj1 = qt.basis(d,0)*qt.basis(d,0).dag()
    proj2 = qt.basis(d,1)*qt.basis(d,1).dag()
    H0 = eps1*proj1 + eps2*proj2 + J*(qt.basis(d,0)*qt.basis(d,1).dag() + qt.basis(d,1)*qt.basis(d,0).dag())
    return {'d':d, 'proj1':proj1, 'proj2':proj2, 'H0':H0}

def build_radical_pair_minimal(B=0.01, J_ex=0.02):
    sx = qt.sigmax(); sy = qt.sigmay(); sz = qt.sigmaz()
    id2 = qt.qeye(2)

    # single-spin ops in tensor space
    sx1 = qt.tensor(sx, id2); sy1 = qt.tensor(sy, id2); sz1 = qt.tensor(sz, id2)
    sx2 = qt.tensor(id2, sx); sy2 = qt.tensor(id2, sy); sz2 = qt.tensor(id2, sz)

    # Hamiltonian
    H_Z = B * (sz1 + sz2)
    H_ex = J_ex * (sx1*sx2 + sy1*sy2 + sz1*sz2)
    H0 = H_Z + H_ex

    # singlet projector
    up = qt.basis(2,0); dn = qt.basis(2,1)
    sing = (qt.tensor(up,dn) - qt.tensor(dn,up)).unit()
    P_S = sing * sing.dag()
    P_S.dims = [[2,2],[2,2]]   # <-- force dims match tensor space

    # triplet projector
    P_T = qt.qeye([2,2]) - P_S

    return {'d':4, 'H0':H0, 'P_S':P_S, 'P_T':P_T}




def build_enzyme_tunneling(epsA=0.0, epsB=0.2, J=0.01):
    d = 3
    e0 = qt.basis(d,0)*qt.basis(d,0).dag()
    e1 = qt.basis(d,1)*qt.basis(d,1).dag()
    e2 = qt.basis(d,2)*qt.basis(d,2).dag()
    H0 = epsA*e0 + epsB*e1 + J*(qt.basis(d,0)*qt.basis(d,1).dag() + qt.basis(d,1)*qt.basis(d,0).dag())
    return {'d':d, 'proj_donor':e0, 'proj_acceptor':e1, 'proj_product':e2, 'H0':H0}


In [3]:
# CELL: entanglement_measures.py - log-negativity & concurrence

def logarithmic_negativity(rho, sys_dims, subsystem=1):
    """
    Compute logarithmic negativity of rho with respect to a bipartition.
    - rho: Qobj density matrix (composite)
    - sys_dims: list of subsystem dimensions, e.g. [2,2] or [2, Nmode]
    - subsystem: index of subsystem to partial-transpose (0 or 1)
    Returns: float (log2 ||rho^{T_sub}||_1)
    """
    # partial transpose via QuTiP
    rho_pt = qt.partial_transpose(rho, sys_dims, subsystem)
    # convert to ndarray and compute trace norm
    mat = rho_pt.full()
    trace_norm = np.sum(np.linalg.svd(mat, compute_uv=False))
    # numerical guard
    trace_norm = max(trace_norm, 1e-12)
    return np.log2(trace_norm)

def concurrence_two_qubit(rho):
    """
    Compute concurrence for a two-qubit density matrix rho (Qobj).
    Implements Wootters' formula.
    """
    # Ensure density matrix numpy array
    rho_mat = rho.full()
    # Pauli-Y
    sy = np.array([[0, -1j], [1j, 0]], dtype=complex)
    YY = np.kron(sy, sy)
    rho_star = np.conjugate(rho_mat)
    R = rho_mat @ YY @ rho_star @ YY
    # eigenvalues of R (non-negative real in ideal)
    vals = np.real(np.linalg.eigvals(R))
    # numerical rounding and sort descending
    vals = np.sort(np.clip(vals, 0, None))[::-1]
    # square roots
    roots = np.sqrt(vals)
    conc = max(0.0, roots[0] - roots[1] - roots[2] - roots[3])
    return float(np.real(conc))


In [4]:
# CELL: choi_and_cptp.py - build finite-time map & Choi test
def build_finite_time_map(H_func, c_ops, d, tlist, solver=qt.mesolve, **solver_kwargs):
    """
    Build finite-time superoperator Phi(t_final, 0) by evolving operator basis.
    - H_func: Hamiltonian (Qobj) or time-dependent function accepted by mesolve
    - c_ops: list of collapse operators (Qobj)
    - d: system dimension
    - tlist: array-like times (we use result at last time)
    Returns:
      - Phi: (d^2, d^2) numpy array superoperator acting on vectorized density matrices
    """
    basis_ops = [qt.basis(d, i) * qt.basis(d, j).dag() for i in range(d) for j in range(d)]
    evolved = []
    tlist = np.array(tlist)
    for B in basis_ops:
        # treat B as initial density-like operator (not necessarily positive)
        result = solver(H_func, B, tlist, c_ops, [])
        rho_t = result.states[-1]
        vec_out = qt.vectorize(rho_t).full().flatten()
        evolved.append(vec_out)
    Phi = np.column_stack(evolved)
    return Phi

def choi_from_superop(Phi, d):
    """
    Convert a superoperator Phi (d^2 x d^2) to Choi matrix (d^2 x d^2)
    using standard reshaping/permutation.
    """
    J = Phi.reshape((d, d, d, d)).transpose((0,2,1,3)).reshape((d*d, d*d))
    return J

def cptp_check_from_choi(J, atol=1e-8):
    """
    Check CPTP from Choi:
    - Positivity: eigenvalues >= -atol
    - Trace-preserving: Tr1[J] == I (within tol)
    Returns (is_cptp, details)
    """
    eigs = np.linalg.eigvals(J)
    min_eig = np.min(np.real(eigs))
    pos_ok = min_eig >= -1e-8
    # partial trace over input (first subsystem): reshape and trace over first index
    d2 = int(np.sqrt(J.shape[0]))
    d = int(np.sqrt(d2))
    J4 = J.reshape((d,d,d,d)).transpose((0,2,1,3))
    tr1 = np.trace(J4, axis1=0, axis2=2)  # yields dxd matrix (numpy)
    # tr1 should equal identity
    I = np.eye(d)
    tp_ok = np.allclose(tr1, I, atol=1e-6, rtol=1e-6)
    return pos_ok and tp_ok, {"min_eig": float(min_eig), "trace1_diff_max": float(np.max(np.abs(tr1 - I)))}


In [5]:
# CELL: vibronic_model.py - two-site + single mode to probe system-environment correlations

def build_two_site_single_mode(J=0.1, eps1=0.0, eps2=0.2, g=0.05, omega=1.0, Nmode=12):
    """
    Builds a composite Hamiltonian:
      H_sys = eps1 |1><1| + eps2 |2><2| + J(|1><2| + h.c.)
      H_mode = omega a^† a
      H_coup = g (|1><1|-|2><2|) ⊗ (a + a^†)   # difference coupling for sensitivity
    Returns:
      - H (Qobj, composite), sys operators and mode operators for diagnostics,
      - dims (sys_dims) for partial transpose and slicing
    """
    # system (two-level) basis
    d_sys = 2
    # Pauli/projectors in system space
    proj1 = qt.basis(d_sys,0)*qt.basis(d_sys,0).dag()
    proj2 = qt.basis(d_sys,1)*qt.basis(d_sys,1).dag()
    sx = qt.sigmax()
    sys_H = eps1*proj1 + eps2*proj2 + J*(qt.tensor(qt.sigmax(), qt.qeye(1)).ptrace(0) if False else J*sx)  # simpler J*sx works for two-level mapping
    # Mode
    a = qt.destroy(Nmode)
    mode_H = omega * qt.tensor(qt.qeye(d_sys), a.dag()*a)
    # System Hamiltonian as composite
    sys_H_comp = eps1 * qt.tensor(proj1, qt.qeye(Nmode)) + eps2 * qt.tensor(proj2, qt.qeye(Nmode)) + J * qt.tensor(sx, qt.qeye(Nmode))
    # Coupling
    coupling = g * qt.tensor(proj1 - proj2, (a + a.dag()))
    H = sys_H_comp + mode_H + coupling
    # Diagnostics operators
    pop1 = qt.tensor(proj1, qt.qeye(Nmode))
    pop2 = qt.tensor(proj2, qt.qeye(Nmode))
    mode_x = qt.tensor(qt.qeye(d_sys), (a + a.dag()))
    mode_n = qt.tensor(qt.qeye(d_sys), a.dag()*a)
    dims = [d_sys, Nmode]
    return {"H": H, "pop1": pop1, "pop2": pop2, "mode_x": mode_x, "mode_n": mode_n, "dims": dims, "a": a}

def mode_quadrature_kurtosis(rho, mode_index=1, trunc_N=None):
    """
    Compute kurtosis of the mode quadrature X = a + a^† for the reduced mode state.
    Kurtosis = E[(X - mu)^4] / (E[(X - mu)^2]^2) - 3  (excess kurtosis)
    """
    # assume rho is composite; reduce to mode
    rho_mode = rho.ptrace(mode_index)
    a = qt.destroy(rho_mode.shape[0])
    X = a + a.dag()
    mu = qt.expect(X, rho_mode)
    m2 = qt.expect(X**2, rho_mode) - mu**2
    m4 = qt.expect((X - mu)**4, rho_mode)
    if m2 == 0:
        return 0.0
    kurt = m4 / (m2**2) - 3.0
    return float(kurt)


In [6]:
# CELL: self_consistent_solver.py - predictor-corrector style integrator (coarse-grained)

def self_consistent_evolve(H_base, O_bio, lambda_fn, lambda_update_fn, c_ops, rho0, tlist, dt=0.01, max_iter_inner=3):
    """
    Evolve rho with H(t) = H_base + lambda(t)*O_bio.
    - lambda_fn: initial function or scalar lambda(0)
    - lambda_update_fn(rho, lambda_prev) -> new_lambda (user-defined feedback)
    This does coarse time-stepping: for each time step:
      - predict lambda(t) using previous value
      - evolve for dt with mesolve using that H
      - recompute lambda from final rho and optionally do a corrector pass
    Returns times, states list
    """
    states = []
    rho = rho0
    lam = float(lambda_fn if np.isscalar(lambda_fn) else lambda_fn(0))
    times = [tlist[0]]
    states.append(rho)
    t = tlist[0]
    for t_next in tlist[1:]:
        # split into internal steps of size dt
        nsteps = max(1, int(np.ceil((t_next - t)/dt)))
        for _ in range(nsteps):
            # predictor H
            H_pred = (H_base + lam * O_bio)
            tsub = [0.0, min(dt, t_next-t)]
            # evolve using mesolve starting from rho (we use operator form)
            result = qt.mesolve(H_pred, rho, tsub, c_ops, [])
            rho_pred = result.states[-1]
            # correct: recompute lambda from rho_pred
            lam_new = lambda_update_fn(rho_pred, lam)
            # simple corrector: average
            lam = 0.5*(lam + lam_new)
            # re-evolve small step using corrected lam
            H_corr = (H_base + lam * O_bio)
            result2 = qt.mesolve(H_corr, rho, [0.0, tsub[-1]], c_ops, [])
            rho = result2.states[-1]
            t = t + tsub[-1]
        times.append(t)
        states.append(rho)
    return np.array(times), states

# Example runner: two-site exciton + simple feedback H_bio = lambda * proj1
def run_example_two_site_with_mode():
    # Build vibronic model
    model = build_two_site_single_mode(J=0.08, eps1=0.0, eps2=0.15, g=0.06, omega=1.2, Nmode=10)
    H_full = model["H"]
    pop1 = model["pop1"]
    pop2 = model["pop2"]
    mode_x = model["mode_x"]
    dims = model["dims"]
    d_sys = dims[0]
    d = d_sys * dims[1]
    # Example collapse: mode damping (to mimic environment relaxation)
    k_mode = 0.02
    c_ops = [np.sqrt(k_mode) * qt.tensor(qt.qeye(d_sys), model["a"])]
    # Base H (exclude H_bio part). For demonstration assume O_bio = pop1 (occupation-dependent energy shift)
    # Extract H_base by removing the projector coupling if present — but simpler: set H_base = H_full - lam0*pop1
    lam0 = 0.0
    O_bio = pop1
    H_base = H_full - lam0 * O_bio
    # initial state: system excited at site 1 and mode vacuum
    rho0_sys = qt.basis(d_sys, 0) * qt.basis(d_sys,0).dag()
    rho0_mode = qt.basis(dims[1], 0) * qt.basis(dims[1], 0).dag()
    rho0 = qt.tensor(rho0_sys, rho0_mode)
    # define lambda feedback: lambda(t) = lambda0 + kappa * Tr[O_bio rho(t)]
    lambda0 = 0.0
    kappa = 0.5
    def lambda_update_fn(rho, lam_prev):
        val = float((rho * O_bio).tr())
        return lambda0 + kappa * val
    # times
    tlist = np.linspace(0, 8.0, 161)
    times, states = self_consistent_evolve(H_base, O_bio, lam0, lambda_update_fn, c_ops, rho0, tlist, dt=0.05)
    # diagnostics: population, mode quadrature avg, kurtosis, log-neg of system+ancilla (we will compute sys-mode entanglement)
    pops1 = [float((s * pop1).tr()) for s in states]
    pops2 = [float((s * pop2).tr()) for s in states]
    mode_x_exp = [float((s * mode_x).tr()) for s in states]
    mode_kurt = [mode_quadrature_kurtosis(s, mode_index=1) for s in states]
    # compute log-negativity between system and mode at final time
    final_rho = states[-1]
    ln_final = logarithmic_negativity(final_rho, sys_dims=dims, subsystem=0)  # partial transpose system (index 0)
    # assemble dataframe
    df = pd.DataFrame({
        "t": times,
        "pop1": pops1,
        "pop2": pops2,
        "mode_x": mode_x_exp,
        "mode_kurtosis": mode_kurt
    })
    # Save CSV and PNG summary
    csv_path = os.path.join(OUTDIR, "vibronic_run.csv")
    df.to_csv(csv_path, index=False)
    # quick plots
    fig, ax = plt.subplots(2,1, figsize=(8,6), tight_layout=True)
    ax[0].plot(times, pops1, label="pop1")
    ax[0].plot(times, pops2, label="pop2")
    ax[0].set_ylabel("Population")
    ax[0].legend()
    ax[1].plot(times, mode_x_exp, label="mode <X>")
    ax[1].plot(times, mode_kurt, label="mode kurtosis (excess)")
    ax[1].set_ylabel("Mode metrics")
    ax[1].legend()
    png_path = os.path.join(OUTDIR, "vibronic_run_summary.png")
    fig.suptitle(f"Vibronic run (kappa={kappa})  ln_final={ln_final:.3f}")
    fig.savefig(png_path, dpi=200)
    plt.show()
    return {"df": df, "ln_final": ln_final, "csv": csv_path, "png": png_path}

# run the example
res = run_example_two_site_with_mode()
print("Saved CSV:", res["csv"])
print("Saved PNG:", res["png"])
print("Final log-negativity (sys|mode):", res["ln_final"])


C:\Users\rhars\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\qutip\solver\solver_base.py:583: FutureWarning: e_ops will be keyword only from qutip 5.3 for all solver
  warnings.warn(


TypeError: float() argument must be a string or a real number, not 'complex'

In [ ]:
# CELL: choi_demo_runner.py - compute finite-time Choi + CPTP check for a 2-level Harsha example

def choi_demo_two_level():
    # simple two-level sample: H = 0.5 * sz + lambda * proj1 with measurement-feedback absent here
    d = 2
    sz = qt.sigmaz()
    proj1 = qt.basis(d,0) * qt.basis(d,0).dag()
    H0 = 0.5 * sz
    # define simple H_bio(t) function to be constant for this test (can be time dependent)
    H_func = H0  # mesolve accepts Qobj as time-independent H
    gamma = 0.15
    c_ops = [np.sqrt(gamma) * qt.sigmam()]
    tlist = np.linspace(0, 1.0, 201)
    Phi = build_finite_time_map(H_func, c_ops, d, tlist, solver=qt.mesolve)
    J = choi_from_superop(Phi, d)
    ok, details = cptp_check_from_choi(J)
    print("CPTP check:", ok, details)
    return ok, details

choi_demo_two_level()


In [ ]:
# CELL 2: Two-site parameter sweep (kappa x gamma), saves CSV + plots
tlist = np.linspace(0,1.0,120)
two = build_two_site()
d = two['d']; proj1 = two['proj1']; H0 = two['H0']

kappas = [0.0, 0.2, 0.5]
gammas = [0.02, 0.05, 0.1]

rows = []
for kappa in kappas:
    for gamma in gammas:
        c_ops = [np.sqrt(gamma) * qt.sigmam()]
        Phi = evolve_basis_self_consistent(H0, proj1, c_ops, tlist, lambda0=0.0, kappa=kappa)
        min_eig, eigs = min_choi_eigenvalue(Phi, d)
        coh_proxy = coherent_information_proxy(Phi, d, samples=48)
        S_traj = entropy_trajectory(H0, proj1, c_ops, tlist, 0.0, kappa, proj1)
        rows.append({
            'system':'two-site','kappa':kappa,'gamma':gamma,
            'min_choi_eig':float(min_eig),'coh_proxy':float(coh_proxy),
            'entropy_final':float(S_traj[-1])
        })

df_two = pd.DataFrame(rows)
two_csv = os.path.join(OUTDIR,'two_site_scan.csv')
df_two.to_csv(two_csv, index=False)
print("Saved two-site CSV:", two_csv)

# example entropy plot (middle case)
example = rows[len(rows)//2]
k_example = example['kappa']; g_example = example['gamma']
c_ops = [np.sqrt(g_example) * qt.sigmam()]
S_traj = entropy_trajectory(H0, proj1, c_ops, tlist, 0.0, k_example, proj1)
plt.figure(figsize=(6,3))
plt.plot(tlist, S_traj, lw=2)
plt.xlabel('Time'); plt.ylabel('Von Neumann entropy S(t)')
plt.title(f'Two-site entropy vs time (kappa={k_example}, gamma={g_example})')
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(OUTDIR,'two_site_entropy_example.png'))
plt.close()
print("Saved entropy plot.")
# Choi heatmap
pivot = df_two.pivot(index='gamma', columns='kappa', values='min_choi_eig')
plt.figure(figsize=(4,3))
plt.imshow(pivot.values, aspect='auto', origin='lower')
plt.colorbar(label='min Choi eigenvalue')
plt.xlabel('kappa'); plt.ylabel('gamma')
plt.xticks(range(len(pivot.columns)), [f'{v:.2f}' for v in pivot.columns])
plt.yticks(range(len(pivot.index)), [f'{v:.3f}' for v in pivot.index])
plt.title('Two-site min Choi eigenvalue heatmap')
plt.tight_layout()
plt.savefig(os.path.join(OUTDIR,'two_site_choi_heatmap.png'))
plt.close()
print("Saved Choi heatmap.")



In [ ]:
# CELL 3: Radical-pair parameter sweep
tlist = np.linspace(0,1.0,120)
rad = build_radical_pair_minimal()
d = rad['d']; H0 = rad['H0']; P_S = rad['P_S']

kappas_r = [0.0, 0.1, 0.3]
gammas_r = [0.01, 0.03, 0.07]

rows = []
for kappa in kappas_r:
    for gamma in gammas_r:
        # pick a simple dissipator as proxy (single-spin lowering on first electron)
        c_ops = [np.sqrt(gamma) * qt.tensor(qt.sigmam(), qt.qeye(2))]
        Phi = evolve_basis_self_consistent(H0, P_S, c_ops, tlist, lambda0=0.0, kappa=kappa)
        min_eig, eigs = min_choi_eigenvalue(Phi, d)
        coh_proxy = coherent_information_proxy(Phi, d, samples=64)
        rows.append({'system':'radical-pair','kappa':kappa,'gamma':gamma,'min_choi_eig':float(min_eig),'coh_proxy':float(coh_proxy)})
df_rad = pd.DataFrame(rows)
rad_csv = os.path.join(OUTDIR,'radical_pair_scan.csv')
df_rad.to_csv(rad_csv, index=False)
print("Saved radical-pair CSV:", rad_csv)


In [ ]:
# CELL 4: Enzyme tunneling example and Lindbladian spectral gap
tlist = np.linspace(0,1.0,120)
en = build_enzyme_tunneling()
d = en['d']; H0 = en['H0']; proj_acceptor = en['proj_acceptor']

k_sink = 0.2
J_sink = np.sqrt(k_sink) * qt.basis(d,2)*qt.basis(d,1).dag()  # product <- acceptor
gamma = 0.05
c_ops = [np.sqrt(gamma) * (qt.basis(d,0)*qt.basis(d,0).dag()), J_sink]

kappa = 0.2
Phi = evolve_basis_self_consistent(H0, proj_acceptor, c_ops, tlist, lambda0=0.0, kappa=kappa)
min_eig, eigs = min_choi_eigenvalue(Phi, d)
gap, eigsL = spectral_gap_of_lindbladian(H0, c_ops)

with open(os.path.join(OUTDIR,'enzyme_example.json'),'w') as f:
    json.dump({'min_choi_eig':float(min_eig),'lindblad_gap':float(gap)}, f)
print("Enzyme example saved; min Choi eig:", float(min_eig), "Lindblad gap:", float(gap))


In [ ]:
# CELL 5: Parameter-fit demo (uses synthetic data from a run with known kappa)
print("Running parameter-fit demo (synthetic)...")
tlist = np.linspace(0,1.0,120)
two = build_two_site()
H0 = two['H0']; proj1 = two['proj1']
true_kappa = 0.3
gamma = 0.05
c_ops = [np.sqrt(gamma) * qt.sigmam()]

# generate synthetic measured acceptor (here we reuse proj1 as observable)
rho = qt.Qobj(proj1)
measured = []
for idx in range(len(tlist)-1):
    dt = tlist[idx+1]-tlist[idx]
    pop = np.real((rho * proj1).tr())
    lam = 0.0 + true_kappa * pop
    Ht = H0 + lam * proj1
    res = qt.mesolve(Ht, rho, [0, dt], c_ops, [])
    rho = res.states[-1]
    measured.append(np.real((rho * proj1).tr()))

res_fit = fit_kappa_from_observables(H0, proj1, c_ops, tlist, proj1, measured, kappa_init=0.1)
print("Fit result:", res_fit)
with open(os.path.join(OUTDIR,'fit_demo.json'),'w') as f:
    json.dump({'fit_result_x': res_fit.x.tolist(), 'fun': float(res_fit.fun)}, f)
print("Saved fit_demo.json")


In [ ]:
# CELL 6: Summary of outputs (print paths)
files = os.listdir(OUTDIR)
print("Output files in", OUTDIR, ":")
for fn in sorted(files):
    print(" -", fn)
print("\nOpen the PNGs/CSVs to inspect results. If any cell throws an error, interrupt and paste the error into chat and I'll help debug.")
